In [2]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt     #pyplot is a collection of functions to make Python work like MATLAB
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

In [3]:
plt.style.use('ggplot')

borders = cfeature.NaturalEarthFeature( #cfeature draws Mapdata from a Database
    category='cultural',
    name='admin_0_countries',
    scale='10m',
    facecolor='none')

graz_lat = 47.0667
graz_lon = 15.45

In [4]:
def readDataset(filePath):
    dataset = Dataset(filePath, mode='r')
    for i in dataset.variables:
        print(i, dataset.variables[i].units, dataset.variables[i].shape)
    lons = dataset.variables['lon'][:,:]
    lats = dataset.variables['lat'][:,:]
    time = dataset.variables['time'][:]
    #title = dataset.title

    try:
        meantmp = dataset.variables['T2M'][:, :]
        meantmp_units = dataset.variables['T2M'].units
        dataset.close()
    except:
        meantmp = dataset.variables['tasmin'][:, :]
        meantmp_units = dataset.variables['tasmin'].units
        dataset.close()

    datasetdict =	{
    "lat_lst": lats,
    "lon_lst": lons,
    "tmp_ary": meantmp,
    "tmp_ut": meantmp_units,
    "time" : time,
    #"title" : title
    }
    return datasetdict

In [5]:
inca_nc_filepath = 'C:/Users/Arthur/Desktop/Daten/INCA_2008_monmean.nc'
incaonspart_nc_filepath = "C:/Users/Arthur/Desktop/Daten/INCA_2008_monmean_SPARTACUS.nc"
spart_nc_filepath = "C:/Users/Arthur/Desktop/Daten/SPARTACUS_2008_monmean.nc"
EOBS_nc_filepath = "C:/Users/Arthur/Desktop/Daten/EOBS_2008_monmean.nc"


In [6]:
inca_monmean = readDataset(inca_nc_filepath)

lon degrees_east (401, 701)
lat degrees_north (401, 701)
time days since 2004-01-01 00:00:00 (12,)
T2M Celsius (12, 401, 701)


In [7]:
incaonspart_monmean = readDataset(incaonspart_nc_filepath)

lon degrees_east (297, 575)
lat degrees_north (297, 575)
time days since 1961-01-01 12:00:00 (12,)
tasmin Celsius (12, 297, 575)


In [8]:
spart_monmean = readDataset(spart_nc_filepath)

lon degrees_east (297, 575)
lat degrees_north (297, 575)
time days since 1961-01-01 12:00:00 (12,)
tasmin Celsius (12, 297, 575)


In [37]:
def naive_fast(latvar,lonvar,lat0,lon0):
    '''
    Find closest point in a set of (lat,lon) points to specified point
    latvar - 2D latitude variable from an open netCDF dataset
    lonvar - 2D longitude variable from an open netCDF dataset
    lat0,lon0 - query point
    Returns iy,ix such that the square of the tunnel distance
    between (latval[it,ix],lonval[iy,ix]) and (lat0,lon0)
    is minimum.
    '''
    # Read latitude and longitude from file into numpy arrays
    latvals = latvar[:]
    lonvals = lonvar[:]
    ny,nx = latvals.shape
    dist_sq = (latvals-lat0)**2 + (lonvals-lon0)**2
    minindex_flattened = dist_sq.argmin()  # 1D index of min element
    minindex_flattened
    iy_min,ix_min = np.unravel_index(minindex_flattened, latvals.shape)
    return iy_min,ix_min

In [38]:
latvar = spart_monmean["lat_lst"]
lonvar = spart_monmean["lon_lst"]
iy, ix = naive_fast(latvar, lonvar, graz_lat, graz_lon)
print('Closest lat lon:', latvar[iy,ix], lonvar[iy,ix])
print('Spartcus Index y =',iy)
print('Spartcus Index x =',ix)
print('Temperature at Index:',spart_monmean["tmp_ary"][0,79,449],'°C')

Closest lat lon: 47.070960998535156 15.44734001159668
Spartcus Index y = 79
Spartcus Index x = 449
Temperature at Index: -1.4451613 °C


In [39]:
latvar = inca_monmean["lat_lst"]
lonvar = inca_monmean["lon_lst"]
iy, ix = naive_fast(latvar, lonvar, graz_lat, graz_lon)
print('Closest lat lon:', latvar[iy,ix], lonvar[iy,ix])
print('INCA Index y =',iy)
print('INCA Index x =',ix)
print('Temperature at Index:',inca_monmean["tmp_ary"][0,134,541],'°C')

Closest lat lon: 47.06629943847656 15.454000473022461
INCA Index y = 134
INCA Index x = 541
Temperature at Index: -1.2419355 °C


In [41]:
latvar = incaonspart_monmean["lat_lst"]
lonvar = incaonspart_monmean["lon_lst"]
iy, ix = naive_fast(latvar, lonvar, graz_lat, graz_lon)
print('Closest lat lon:', latvar[iy,ix], lonvar[iy,ix])
print('INCA on SPARTACUS Index y =',iy)
print('INCA on SPARTACUS Index x =',ix)
print('Temperature at Index:',inca_monmean["tmp_ary"][0,134,541],'°C')

Closest lat lon: 47.070960998535156 15.44734001159668
INCA on SPARTACUS Index y = 79
INCA on SPARTACUS Index x = 449
Temperature at Index: -1.2419355 °C
